<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/Jaccard_Similarity_Recommender/MovieLens_JS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> Pearson similarity Recommender</font></h1>

---

<center>
  <img src="https://bobliu.io/assets/img/cards.509a5045.jpg" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en Pearson Similarity.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

## 1. Contexto


El conjunto de datos MovieLens es uno de los conjuntos de datos de recomendación más populares y ampliamente utilizados en la investigación de sistemas de recomendación. Fue creado por el GroupLens Research Project en la Universidad de Minnesota para impulsar la investigación en sistemas de recomendación, proporcionando un recurso valioso para la comunidad académica y promoviendo el desarrollo y la comprensión de tecnologías de recomendación personalizada.


<b>Descripción de datos</b>

El conjunto de datos MovieLens contiene información sobre:

<b>Películas:</b> Detalles sobre las películas, incluyendo su título, género y año de lanzamiento.

<b>Usuarios:</b> Perfiles de los usuarios que han calificado y/o etiquetado las películas, incluyendo su ID y otros detalles demográficos opcionales.

<b>Calificaciones:</b> Calificaciones numéricas que los usuarios asignan a las películas en una escala de 1 a 5.

<b>Etiquetas:</b> Palabras clave o tags proporcionados por los usuarios para describir el contenido o la esencia de las películas.

El conjunto de datos es ampliamente utilizado con fines académicos y de investigación, siendo una referencia en el diseño y evaluación de sistemas de recomendación de películas. También es útil para el análisis de tendencias y comportamientos en la visualización de películas y la interacción del usuario con el contenido.

<strong>Puede consultar este [link](https://grouplens.org/datasets/movielens/) para leer más sobre la fuente de datos MovieLens proporcionada por GroupLens Research en la Universidad de Minnesota.</strong>

## 2. Descargar y preparar Dataset

In [375]:
# Descargar el dataset Movielens
#!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
#!unzip dataset.zip
#!ls -la

In [376]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [377]:
links   = pd.read_csv("ml-latest-small/links.csv")
movies  = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")
tags    = pd.read_csv("ml-latest-small/tags.csv")


In [ ]:
links.head()

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
tags.head()

In [382]:
print("  Movies: {} \n  Ratings: {}".format(len(movies), len(ratings)))


  Movies: 9742 
  Ratings: 100836


In [383]:
# Fusiona ambos datasets basados en la columna 'movieId'
data = pd.merge(ratings, movies, on='movieId')

In [384]:
movie_titles = data['title'].unique().tolist()
movie_ids = data['movieId'].unique().tolist()


In [385]:
# Convertir el dataset de calificaciones a un formato binario
data['binary_rating'] = data['rating'].apply(lambda x: 1 if x >= 4 else 0).astype(int)


In [386]:
# Crear matriz pivotada de usuarios y películas con el formato binario
user_movie_binary_rating = data.pivot_table(index='userId', columns='title', values='binary_rating')


In [387]:
#500 películas más vistas
movies_pop = user_movie_binary_rating.isnull().sum().sort_values()[:500]


In [388]:
user_movie_binary_rating = user_movie_binary_rating[movies_pop.index.tolist()]
user_movie_binary_rating = user_movie_binary_rating.reset_index()

In [389]:
# Llenar los valores NaN con 0
user_movie_binary_rating = user_movie_binary_rating.fillna(0)

Muestreo

In [390]:
from sklearn.model_selection import train_test_split

# Convertir la matriz pivotada en un DataFrame y dividir en train y test
train_data, test_data = train_test_split(user_movie_binary_rating, test_size=0.2, random_state=42)


In [391]:
# Volver a crear matrices pivotadas para entrenamiento y prueba
train_data_matrix = train_data.fillna(0)
test_data_matrix = test_data.fillna(0)

In [ ]:
test_data_matrix.head()

## 3. Jaccard Similarity

Aplicaremos el enfoque de similaridad de pearson con enfoque ítem-ítem

In [393]:
data['rating'] = data['rating'].astype(float)

In [394]:
from sklearn.metrics.pairwise import pairwise_distances

# Convertir DataFrame a matriz NumPy
binary_train_data_numpy = train_data_matrix.values

# Calcular la matriz de similaridad de Jaccard
jaccard_distances = pairwise_distances(binary_train_data_numpy.T, metric='jaccard')  # T para ítem-ítem
jaccard_similarity = 1 - jaccard_distances
jaccard_df = pd.DataFrame(jaccard_similarity, index=train_data_matrix.columns, columns=train_data_matrix.columns)
jaccard_df = jaccard_df.reset_index()

In [ ]:
pd.set_option('display.max_colwidth', None)

# Seleccionar un usuario (por ejemplo, el usuario con ID 5)
user_id = 83

# Películas que el usuario ha visto
user_movies = data[data['userId'] == user_id]
watched_movies = user_movies[['title', 'genres', 'rating', 'binary_rating']]
watched_movies.sort_values(by = 'rating', ascending = False, inplace = True)

print(f"Películas vistas por el usuario {user_id}:")
watched_movies.head(5)

In [ ]:
# Porque te gustó "Truman Show, The (1998)" ....
jaccard_df[['title', 'Truman Show, The (1998)']].sort_values(by = 'Truman Show, The (1998)', ascending = False)[1:].head(3)

In [ ]:
# Porque te gustó "Terminator 2" ....
jaccard_df[['title', 'Inception (2010)']].sort_values(by = 'Inception (2010)', ascending = False)[1:].head(3)

In [ ]:
# Porque te gustó "Indiana Jones and the Temple of Doom" ....
jaccard_df[['title', 'Dark Knight, The (2008)']].sort_values(by = 'Dark Knight, The (2008)', ascending = False)[1:].head(3)

In [399]:
%%capture
!pip install adjustText

PCA para entender la similaridad

In [400]:
# Crear matriz pivotada de usuarios y películas
user_movie_rating = data.pivot_table(index='userId', columns='title', values='rating')

#30 películas más vistas
movies_pop = user_movie_rating.isnull().sum().sort_values()[:50]

user_movie_rating = user_movie_rating[movies_pop.index.tolist()]
user_movie_rating = user_movie_rating.reset_index()

movie_correlation = user_movie_rating.fillna(0).corr()
movie_correlation = movie_correlation.reset_index()


In [ ]:
from sklearn.decomposition import PCA
from adjustText import adjust_text

# Aplicar PCA
pca = PCA(n_components=2)
movie_pca = pca.fit_transform(jaccard_df.drop(columns=['title', 'userId']))

# Obtener las n películas más vistas
top_30_movies = user_movie_rating.count().sort_values(ascending=False)[:50].index.tolist()
top_30_movies = list(set(top_30_movies)-set(['title', 'userId']))

# Graficar las n películas más vistas usando PCA
plt.figure(figsize=(20, 10))
for title in top_30_movies:
    idx = movie_correlation[movie_correlation['title'] == title].index[0]
    plt.scatter(movie_pca[idx, 0], movie_pca[idx, 1], s=50)
    plt.text(movie_pca[idx, 0], movie_pca[idx, 1], title, fontsize=7)

# Evitar superposición de etiquetas (si tienes adjustText instalado)
#adjust_text(texts, arrowprops=dict(arrowstyle='->', color='red'))

plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Visualización de la Similaridad de las 30 Películas Más Vistas usando PCA')
plt.show()

## 4. Evaluación del modelo

---
## Gracias por completar este laboratorio!